In [1]:
import pandas as pd
import datetime
import requests
import bs4
from bs4 import BeautifulSoup, Comment
import re
import cPickle
import time
import os
from matplotlib import pyplot as plt

%matplotlib inline

In [3]:
URL = 'https://www.viamichelin.com'
global_extension = '/web/Restaurants?geoboundaries=\
-41.7713117,-156.09375:77.389504,128.3203125&page='

paths = []
start = datetime.datetime.now()
print 'start time: %s\n' %start

for page in range(1, 2): 
    pre_mich = requests.get(URL + global_extension + str(page))
    pre_soup = BeautifulSoup(pre_mich.content, "lxml")
    for restaurant in pre_soup.find_all('a', href=re.compile('/web/Restaurant/')):
        path = restaurant["href"]
        paths.append(path)

print '\ndone!'
paths = pd.DataFrame(paths)
paths.columns = ['path']
finish = datetime.datetime.now()
print '\nfinish time: %s' %finish
print '\ntime elapsed: %s' %(finish - start)

start time: 2017-01-26 17:11:14.480305


done!

finish time: 2017-01-26 17:11:15.517675

time elapsed: 0:00:01.037370


In [4]:
paths

path
0   /web/Restaurant/Cruis-04230-Auberge_de_l_Abbay...
1   /web/Restaurant/Cuq_Toulza-81470-Cuq_en_Terras...
2    /web/Restaurant/Deauville-14800-1899-11005-41102
3   /web/Restaurant/Delme-57590-A_la_XIIe_Borne-11...
4   /web/Restaurant/Divonne_les_Bains-01220-Chatea...
5   /web/Restaurant/Domme-24250-L_Esplanade-11046-...
6   /web/Restaurant/Douarnenez-29100-Le_Clos_de_Va...
7   /web/Restaurant/Eguisheim-68420-Auberge_des_Tr...
8   /web/Restaurant/Le_Fel-12140-Auberge_du_Fel-11...
9   /web/Restaurant/Biarritz-64200-Le_Clos_Basque-...
10  /web/Restaurant/Biarritz-64200-Chez_Albert-111...
11  /web/Restaurant/Evian_les_Bains-74500-La_Verni...
12  /web/Restaurant/Biarritz-64200-Philippe-1112-4...
13  /web/Restaurant/Evosges-01230-L_Auberge_Campag...
14  /web/Restaurant/Eze-06360-La_Chevre_d_Or-11132...
15  /web/Restaurant/Fayence-83440-L_Escourtin-1114...
16  /web/Restaurant/La_Ferte_sous_Jouarre-77260-Le...
17  /web/Restaurant/Cocures-48400-La_Lozerette-111...
18  /web/Restaurant/Mousterlin-29170-L_Intemporel-...
19  /web/Restaurant/La_Garde_Adhemar-26700-Le_Logi...
20  /web/Restaurant/Gerardmer-88400-Cote_Lac-11226...
21  /web/Restaurant/Gordes-84220-Les_Bories-11254-...
22  /web/Restaurant/Gourdon-46300-Hostellerie_de_l...
23  /web/Restaurant/Gramat-46500-Lion_d_Or-11271-4...

In [7]:
URL = 'https://www.viamichelin.com'
world_soup = []


start = datetime.datetime.now()
print 'start time: %s' %start

for i in range(len(paths)): ## first part out of 18670
    time.sleep(1)
    tiny_request = requests.get(URL + paths['path'][i])
    if str(tiny_request) != '<Response [200]>':
        print '\n'
        print URL + paths['path'][i]
        print '\n'
    tiny_soup = BeautifulSoup(tiny_request.content, "lxml")
    world_soup.append(tiny_soup)
    
midway = datetime.datetime.now()
print '\n\nall done!  that took %s.  pickle time...' %(midway - start)

finish = datetime.datetime.now()
print '\nfinish time: %s' %finish
print '\nelapsed time: %s' %(finish - start)
os.system('say "Finished!"')

start time: 2017-01-26 17:13:00.362197


all done!  that took 0:00:40.118525.  pickle time...

finish time: 2017-01-26 17:13:40.480888

elapsed time: 0:00:40.118691


0

In [9]:
world_soup[0]

<!DOCTYPE html>\n<html lang="en" xmlns:v="urn:schemas-microsoft-com:vml">\n<head>\n<meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n<meta charset="unicode-escape"/>\n<meta content="#1095f9" name="theme-color"/>\n<title>Auberge de l'Abbaye\xa0- Peyruis\xa0:\xa0a Michelin Guide restaurant</title>\n<meta content="Auberge de l'Abbaye: Michelin Guide review, users review, type of cuisine, opening times, meal prices .." name="description"/>\n<meta content="width=device-width, initial-scale=1.0, user-scalable=no" name="viewport"/>\n<meta content="yes" name="apple-mobile-web-app-capable"/>\n<meta content="app-id=443142682" name="apple-itunes-app"/>\n<meta content="app-id=com.viamichelin.android.viamichelinmobile" name="google-play-app"/>\n<meta content="https://www.viamichelin.com/logo-social.png" property="og:image"/>\n<meta content="Auberge de l'Abbaye\xa0- Peyruis\xa0:\xa0a Michelin Guide restaurant" property="og:title"/>\n<meta content="Auberge de l'Abbaye: Michelin Guide review, us

In [10]:
def restaurantify(list_soup):
    
    '''This function creates a dataframe with restaurant observations and columns \
     containing relevant information.'''
    
    name = []
    cuisine = []
    blurb = []
    high_price = []
    low_price = []
    distinctions = []
    photos = []
    photo_count = []
    add_info = []
    latitude = []
    longitude = []
    comments = []
    num_comments = []
    special_comment = []
    currency = []
    
    count = -1
    
    now = datetime.datetime.now()
    print 'restaurantify start time: %s' %now

    
    for tiny_soup in list_soup:
        
        ## GET RESTAURANT NAME
        name_result = tiny_soup.findAll('div', class_ = 'datasheet-item datasheet-name')
        if name_result:
            for rest_name in name_result:
                name_text = rest_name.text.encode('utf-8')
                name_text = name_text.replace('\n\t', '')
                name_text = name_text.replace('\n', '')
                name.append(name_text)
        else:
            name.append(None)
              
        ## GET RESTAURANT CUISINE
        cuisine_result = tiny_soup.findAll('div', class_ = 'datasheet-cooking-type')
        if cuisine_result:
            for rest_cuisine in cuisine_result:
                lil_cuisine = rest_cuisine.text.encode('utf-8')
                lil_cuisine = lil_cuisine.replace('Cuisine', '')
                lil_cuisine = lil_cuisine.replace('cuisine', '')
                cuisine.append(lil_cuisine)
        else:
            cuisine.append(None)
                
        ## GET BLURB
        blurb_result = tiny_soup.findAll('div', class_ = 'datasheet-description')
        if blurb_result:
            for rest_blurb in blurb_result:
                for j in rest_blurb.find_all('blockquote'):
                    text = j.text.encode('utf-8')
                    text = text.replace('\n\t\t\t', '')
                    text = text.replace('\t', '')
                    blurb.append(text)
        else:
            blurb.append(None)
            

        ## GET PRICE RANGE
        price_result = tiny_soup.findAll('div', class_ = 'datasheet-price')
        if price_result:
            for rest_price in price_result:
                price_list = []
                for j in rest_price.findAll('em'):
                    price_list.append(j)
                if len(price_list) == 2:
                    low = price_list[0].text.encode('utf-8')
                    low = low.split(' ')
                    currency.append(low[1])
                    try:
                        low_price.append(int(low[0]))
                    except:
                        low_price.append(low[0])
                    high = price_list[1].text.encode('utf-8')
                    high = high.split(' ')
                    try:
                        high_price.append(int(high[0]))
                    except:
                        high_price.append(high[0])
                else:
                    currency.append(None)
                    low_price.append(None)
                    high_price.append(None)
        else:
            high_price.append(None)
            low_price.append(None)
            currency.append(None)
            
        
        ## GET DISTINCTIONS
        distinct_result = tiny_soup.findAll('div', class_ = 'datasheet-quotation')
        if distinct_result:
            for big_distinction in distinct_result:
                distinction_list = []
                for lil_distinction in big_distinction.findAll('span'):
                    distinction_list.append(lil_distinction['class'])
                distinctions.append(distinction_list)
        else:
            distinctions.append(None)
            
        
        ## PHOTO COUNTS
        photocount_result = tiny_soup.findAll('section', class_ = 'datasheet')
        if photocount_result:
            for big_mess in photocount_result:
                mess = big_mess.find('span', class_ = 'icon-photo')
                if mess:
                    photo_count.append(int(mess.text))  # hasn't been run w 'int()' yet
                else:
                    photo_count.append(0)
        else:
            photo_count.append(0)
        
        
        ## GET PHOTO URLs
        photo_URLs = []
        photo_result = tiny_soup.findAll('div', class_ = \
                                         'datasheet-more-info datasheet-photo clearfx')
        if photo_result:
            for urls in photo_result:
                if urls:
                    url_result = urls.findAll('img')
                    if url_result:
                        for url in url_result:
                            lil_url = url['src']
#                            lil_url = lil_url.replace('https://download.viamichelin.com', '')
                            photo_URLs.append(lil_url)
                        photos.append(photo_URLs)
                    else:
                        photos.append([])
                else:
                    photos.append([])
        else:
            photos.append([])

            
        ## GET MESSY ADDITIONAL INFO, HAVE FUN DEALING WITH IT LATER 
        infos = []
        big_info = tiny_soup.findAll('div', class_ = 'datasheet-more-info clearfx')
        if big_info:
            for rest_add_info in big_info:
                for advice in rest_add_info.findAll('li'):
                    if advice:
                        infos.append(advice.text.encode('utf-8'))
                    else:
                        infos.append(None)
                        add_info.append(None) ## NEW
            add_info.append(infos)
        else:
            add_info.append(None)
        
        
        ## GET LATITUDE AND LONGITUDE 
        geolocs = tiny_soup.findAll('div', class_ = 'datasheet-link-to-itinerary')
        if geolocs:
            for lil_latlon in geolocs:
                for link in lil_latlon.findAll('a'):
                    latlon_split = link['href']
                    latlon_split = (latlon_split.split(';'))[1]
                    latlon_split = latlon_split.split(':')
                    latitude.append(float(latlon_split[1]))
                    longitude.append(float(latlon_split[0]))
        else:
            latitude.append(None)
            longitude.append(None)
            
        
        ## GET COMMENT LIST
        comment_mess = tiny_soup.find_all(string=lambda text:isinstance(text, Comment))
        if comment_mess:
            comment_list = []
            for comment in comment_mess:
                x = comment.extract()
                comment_list.append(x)
            comments.append(comment_list)
            num_comments.append(len(comment_list))
            special_comment.append(comment_list[5])
        else:
            comments.append(None)  
            num_comments.append(0)
            special_comment.append(None)
            
            
        ## MAKE SURE NO MISMATCHES IN LIST LENGTH    
        length_list = [len(name), len(cuisine), len(blurb), len(high_price), len(low_price), \
                   len(distinctions), len(photos), len(photo_count), len(add_info),\
                  len(latitude), len(longitude), len(comments), len(num_comments), \
                   len(special_comment), len(currency)]
        
        if len(set(length_list)) > 1:
            
            print '\n'
            print set(length_list)
            print 'Things got screwy near %s!\n' %name_text
            print 'name: %s' %len(name)
            print 'cuisine: %s' %len(cuisine)
            print 'distinctions: %s' %len(distinctions)
            print 'blurb: %s' %len(blurb)
            print 'high_price: %s' %len(high_price)
            print 'low_price: %s' %len(low_price)  
            print 'photos: %s' %len(photos)
            print 'photo_count: %s' %len(photo_count)
            print 'add_info: %s' %len(add_info)
            print 'latitude: %s' %len(latitude)
            print 'longitude: %s' %len(longitude)
            print 'comments: %s' %len(comments)
            print 'num_comments: %s' %len(num_comments)
            print 'special_comment: %s' %len(special_comment)
            os.system('say "Uh oh, something\'s wrong."')
            break
        
        else:
            continue
            
        count += 1
        if count%100 == 0:
            print float(count)/len(list_soup),
            print 'line: %s' %count,
            
    ## GET WORD COUNT FOR BLURB
    count2 = 0
    num_words = []
    print '\n'
    for blurby in blurb:
        if blurby:
            x = blurby.split(' ')
            countx = len(x)
            num_words.append(countx)
        else:
            num_words.append(0)
        if float(count2)%1000 == 0:
            print count2
            count2 += 1

    print '\n'
    print 'name: %s' %len(name)
    print 'cuisine: %s' %len(cuisine)
    print 'distinctions: %s' %len(distinctions)
    print 'blurb: %s' %len(blurb)
    print 'high_price: %s' %len(high_price)
    print 'low_price: %s' %len(low_price)  
    print 'photos: %s' %len(photos)
    print 'photo_count: %s' %len(photo_count)
    print 'add_info: %s' %len(add_info)
    print 'latitude: %s' %len(latitude)
    print 'longitude: %s' %len(longitude)
    print 'comments: %s' %len(comments)
    print 'num_comments: %s' %len(num_comments)
    print 'special_comment: %s' %len(special_comment)
    print 'num_words: %s' %len(num_words)
    
    df = pd.DataFrame(zip(name, cuisine, blurb, num_words, high_price, low_price, currency, \
distinctions, photos, photo_count, add_info, latitude, longitude, comments, num_comments, \
special_comment), columns = ['name', 'cuisine', 'blurb', 'num_words', 'high_price', \
'low_price', 'currency', 'distinctions', 'photos', 'photo_count', 'add_info', 'latitude', \
'longitude', 'comments', 'num_comments', 'special_comment'])
    
    print '\n'
    now_now = datetime.datetime.now()
    print 'restaurantify end time: %s' %now_now
    elapsed_now = now_now - now
    print 'restaurantify elapsed time: %s' %elapsed_now
    
    return df